In [1]:
import pandas as pd
import numpy as np

# format sig figs
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Append Observed to Modeled (previously completed)
## Add the script here...

In [2]:
#############################
# Load the observed records
# 
obs = pd.read_csv(r'R:\FastTrips\CHTS and OBS ft_output\OBS_ft_output_w_stops.csv')
obs.fillna('',inplace=True)
obs['person_id'] = obs['person_id'].apply(lambda x: x.split("_")[-1]).astype('str')
obs['A_id_num'] = obs['A_id_num'].replace("",0)
obs['A_id_num'] = obs['A_id_num'].astype('int')
obs = obs.drop('trip_list_id_num',axis=1)

# ADD THE PROPER trip_list_id_num
obsinput = pd.read_csv(r'J:\Projects\FasTrips\obs\input\1.0\OBS_fasttrips_demand_v1.0\trip_list.txt')
obsinput['person_id'] = obsinput['person_id'].astype('int').astype('str')

obs = pd.merge(obs, obsinput[['person_id','trip_list_id_num']], on='person_id')
obs.to_csv('R:\FastTrips\CHTS and OBS ft_output\OBS_ft_output_w_stops_BN.csv')
# Make sure observed matches modeled


##########################
# On Board Survey Results
model = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\chosenpaths_links.csv')
model['route_id'].fillna("",inplace=True)
# FOR NOW, assume we look at only the last iteration of path sets ?
model = model[model['iteration'] == model['iteration'].max()]
model['person_id'] = model['person_id'].astype('int').astype('str')


#########################
# Pathsets
pathset_links = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\pathset_links.csv')
pathset_links['route_id'].fillna("",inplace=True)
pathset_links = pathset_links[pathset_links['iteration'] == pathset_links['iteration'].max()]

# For OBS
pathset_paths = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\pathset_paths.csv')


In [3]:
# Load data CHTS

# # For testing, use a known list of trips, e.g., fast trips output
# obs = pd.read_csv(r'J:\Projects\FasTrips\chts\output\CHTS_fasttrips_demand_v0.1_stochastic_iter2_nocap\chosenpaths_links.csv')
# # Won't need to do this with real observed data
# obs['route_id'].fillna("",inplace=True)
# obs = obs[obs['iteration'] == obs['iteration'].max()]

# CHTS
# Load the model records
# model = pd.read_csv(r'J:\Projects\FasTrips\chts\output\CHTS_fasttrips_demand_v0.1_stochastic_iter2_nocap\chosenpaths_links.csv')
# model['route_id'].fillna("",inplace=True)
# # FOR NOW, assume we look at only the last iteration of path sets ?
# model = model[model['iteration'] == model['iteration'].max()]

# Load pathset links
## CHTS
# pathset_links = pd.read_csv(r'J:\Projects\FasTrips\tableau\path-set\pathset_links.csv')
# pathset_links['route_id'].fillna("",inplace=True)
# pathset_links = pathset_links[pathset_links['iteration'] == pathset_links['iteration'].max()]

# Reset Indeces

In [4]:
obs.reset_index(inplace=True)
model.reset_index(inplace=True)
pathset_links.reset_index(inplace=True)
pathset_paths.reset_index(inplace=True)

In [5]:
# 

# Create Unique ID

In [6]:
obs['unique_id'] = obs['person_id'].astype('str') +"_"+obs['trip_list_id_num'].astype('str')
model['unique_id'] = model['person_id'].astype('str') +"_"+model['trip_list_id_num'].astype('str')
pathset_links['unique_id'] = pathset_links['person_id'].astype('int').astype('str') +"_"+pathset_links['trip_list_id_num'].astype('str')
pathset_paths['unique_id'] = pathset_paths['person_id'].astype('str') +"_"+pathset_paths['trip_list_id_num'].astype('str')
pathset_paths['person_id'] = pathset_paths['person_id'].astype('int').astype('str')
pathset_paths['unique_id'] = pathset_paths['person_id'].astype('str') +"_"+ pathset_paths['trip_list_id_num'].astype('str')

** each unique ID should exist within the observed and the model results **

In [7]:
# Compare the unique_id fields between each data set
print len(obs.groupby('unique_id').count().index)
print len(model.groupby('unique_id').count().index)
print len(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))

22951
20836
18131


Some observed trips were filtered out, but the number of modeled trips should be the same as the 
number of common unique_ids. For some reason it's less

In [8]:
# Use the common unique_ids to filter out the data
model = model[model['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))]
obs = obs[obs['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))]

In [9]:
print len(obs.groupby('unique_id').count().index)
print len(model.groupby('unique_id').count().index)
print len(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))

18131
18131
18131


**Compare against the pathset links and paths files**

In [10]:
print len(obs.groupby('unique_id').count().index)
print len(pathset_links.groupby('unique_id').count().index)
print len(list(set((obs['unique_id'].values)).intersection((pathset_links['unique_id'].values))))
obs = obs[obs['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))]
pathset_links = pathset_links[pathset_links['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))]

18131
25348
18131


In [11]:
print len(obs.groupby('unique_id').count().index)
print len(pathset_paths.groupby('unique_id').count().index)
print len(list(set((obs['unique_id'].values)).intersection((pathset_paths['unique_id'].values))))
obs = obs[obs['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((pathset_paths['unique_id'].values))))]
pathset_paths = pathset_paths[pathset_paths['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((pathset_paths['unique_id'].values))))]

18131
25441
18131


# Join correct transit route IDs to OBS data

# Add a transit agency lookup too

In [12]:
def produce_path_fields(df, group):
    '''
    '''
    # create "path_routes"
    df['path_routes'] = df['route_id'].apply(lambda x: x.strip())
    path_routes = pd.DataFrame(df.groupby(group)['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    result_df = pd.DataFrame(index=path_routes.index)
    result_df['path_routes'] = path_routes
    
    # create "path_modes"
    df['path_modes'] = df['mode'].apply(lambda x: x.strip())
    result_df['path_modes'] = pd.DataFrame(df.groupby(group)['mode'].apply(lambda x: "%s" % ' '.join(x).strip()))

    # Create "path_components"
    df['path_components'] = df['A_id'].astype('str')+" "+df['mode']+" "+df['route_id'] +"_"+ df['B_id'].astype('str')
    df['path_components'] = df['path_components'].apply(lambda x: x.strip())
    result_df['path_components'] = pd.DataFrame(df.groupby(group)['path_components'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    # Return ID field from index
    result_df['unique_id'] = result_df.index.get_level_values(0).values
    

    return result_df

# Produce joined fields from pathset link files 

In [13]:
# Create caluclated joined fields for the observed, modeled, and pathset links file

# Concatenate modes, route IDs, etc to produce unique trip identities
# Do this for each set of trips in the oberved data, as well as for the modeled, since we don't have the field produced by FT
observed_path = produce_path_fields(obs, group=['unique_id'])

In [14]:
modeled_path = produce_path_fields(model, group=['unique_id'])

In [15]:
# also process the detailed pathset_links files, so each path in the pathset has a unique trip identity
new_pathset = produce_path_fields(pathset_links, group=['unique_id','pathnum'])

### Weirdness with unique_id mismatch between model/obs and the pathset_links file

In [16]:


################################
# For some reason there is not a smooth overlap between the modeled pathset_link unique_ids and the model/observed data
# there are unique IDs missing from each, so I guess we just take an intersection of what's available for all of them?
print len(new_pathset)
print len(model)
print len(obs)

305995
94335
70988


In [17]:
pathset_links['unique_id'] == ''

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
...
3533767    False
3533768    False
3533769    False
3533770    False
3533771    False
3533772    False
3533773    False
3533774    False
3533775    False
3533776    False
3533777    False
3533778    False
3533779    False
3533780    False
3533781    False
Name: unique_id, Length: 2196541, dtype: bool

In [18]:
obs = obs[obs['unique_id'].isin(new_pathset['unique_id'].values)]
model = model[model['unique_id'].isin(new_pathset['unique_id'].values)]
new_pathset = new_pathset[new_pathset['unique_id'].isin(obs['unique_id'].values)]
new_pathset = new_pathset[new_pathset['unique_id'].isin(model['unique_id'].values)]

In [19]:
print len(new_pathset)
print len(model)
print len(obs)

305995
94335
70988


## Compare if modeled/observed trips match, completed or partially

In [20]:
# Join the observed and modeled fields
df = pd.merge(observed_path, modeled_path, on='unique_id',suffixes=("_observed","_model"))

In [21]:
# Find rows with matching path routes
complete_route_match = df[df['path_routes_observed'] == df['path_routes_model']]
complete_mode_match = df[df['path_modes_observed'] == df['path_modes_model']]
# Add complete_agency_match when available
# complete_route_match = df[df['path_agency_observed'] == df['path_agency_model']]

In [22]:
## Extract order of transit routes taken
df['model_path_route_list'] = df['path_routes_model'].apply(lambda x: x.split(" "))
df['obs_path_route_list'] = df['path_routes_observed'].apply(lambda x: x.split(" "))

df['model_path_mode_list'] = df['path_modes_model'].apply(lambda x: x.split(" "))
df['obs_path_mode_list'] = df['path_modes_observed'].apply(lambda x: x.split(" "))

In [23]:
# Isolate transit modes only, because almost all trips have walk & transfer components
non_transit_modes = ['transfer','walk_access','walk_egress','bike_access','bike_egress',
                     'PNR_access','PNR_egress','KNR_access','KNR_egress']
df['model_transit_modes'] = df['model_path_mode_list'].apply(
    lambda row: [element for element in row if element not in non_transit_modes])
df['obs_transit_modes'] = df['obs_path_mode_list'].apply(
    lambda row: [element for element in row if element not in non_transit_modes])

In [24]:
# df['routes_in_path'].apply(lambda x: len(x.split(" ")))

# Get a 

# Routes
df.apply(lambda row: all(i in row['model_path_route_list'] for i in row['obs_path_route_list']), axis=1)
df['routes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_route_list'], df['obs_path_route_list'])]

# All Modes
df.apply(lambda row: all(i in row['model_path_mode_list'] for i in row['obs_path_mode_list']), axis=1)
df['all_modes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_mode_list'], df['obs_path_mode_list'])]

# Transit modes only
df.apply(lambda row: all(i in row['model_path_mode_list'] for i in row['obs_path_mode_list']), axis=1)
df['transit_modes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_mode_list'], df['obs_path_mode_list'])]


# Agency Intersection

# All path components


# Exact Match of Path Routes, Modes, Components

In [25]:
# Exact match of path modes
complete_mode_match['complete_mode_match'] = 1
df = pd.merge(df, complete_mode_match[['unique_id','complete_mode_match']], how='left', on='unique_id')
df['complete_mode_match'].fillna(0,inplace=True)

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [26]:
complete_route_match['complete_route_match'] = 1

In [27]:
df = pd.merge(df, complete_route_match[['unique_id','complete_route_match']], how='left', on='unique_id')

In [28]:
df['complete_route_match'] = df['complete_route_match'].replace('nan',0)


In [29]:
df['complete_route_match'].mean()

0.0

In [30]:
df['complete_mode_match'].mean()

0.2227124813854724

## Export fields to CSV for Tableau:
- 'complete_route_match' 0/1
- 'complete_mode_match' 0/1

# % trips with matching or partial matching routes

In [31]:
# Now we find the percent of trips with matching routes or partial matching routes
# df.mean()[['complete_match','partial_match']]
# Note, shuold be 100% when using the same data set, what's up??

#############
# Join the filtered data to the original results
df['common_mode_count'] = [len(row) for row in df['all_modes_intersection']]
df['common_transit_mode_count'] = [len(row) for row in df['transit_modes_intersection']]

# How many rows have at least one mode in common?
df['partial_mode_match'] = [1 if row > 0 else 0 for row in df['common_mode_count']]
df['partial_transit_mode_match'] = [1 if row > 0 else 0 for row in df['common_transit_mode_count']]

In [32]:

df['partial_mode_match'].mean()

0.9986763002592245

In [33]:
df['partial_transit_mode_match'].mean()

0.9986763002592245

## Export fields to CSV for Tableau:
- 'partial_mode_match' 0/1
- 'complete_mode_match' 0/1

# Compare probability of observed path to pathset


In [68]:
observed_path.head(3)

,path_routes,path_modes,path_components,unique_id
unique_id,,,,
100066_15335,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335
1000_431,MUNI Caltrain_San Francisco Burlingame Tri-...,walk_access local_bus transfer commuter_rail w...,930.0 walk_access _24893.0 24893.0 local_bus M...,1000_431
1000_432,MUNI Caltrain_San Francisco Burlingame Tri-...,walk_access local_bus transfer commuter_rail w...,930.0 walk_access _24893.0 24893.0 local_bus M...,1000_432


In [69]:
new_pathset.head(3)

path_routes                         path_modes  \
unique_id    pathnum                                                  
100066_15335 0          76_14MNO  walk_access local_bus walk_egress   
             1          74_44NBL  walk_access local_bus walk_egress   
             2          74_62NBL  walk_access local_bus walk_egress   

                                                        path_components  \
unique_id    pathnum                                                      
100066_15335 0        2361 walk_access _9376 9376 local_bus 76_14MNO...   
             1        2361 walk_access _9376 9376 local_bus 74_44NBL...   
             2        2361 walk_access _9376 9376 local_bus 74_62NBL...   

                         unique_id  
unique_id    pathnum                
100066_15335 0        100066_15335  
             1        100066_15335  
             2        100066_15335

In [78]:
## Add a field to the new_pathset that lists the pathnum
new_pathset['pathnum'] = new_pathset.index.get_level_values(1)

In [79]:
# Do this with a merge?

df = pd.merge(observed_path, new_pathset, how='left',
              left_on=['unique_id','path_modes'],right_on=['unique_id','path_modes'], suffixes=['_obs','_pathset'])

In [80]:
print len(df)
print len(observed_path)
print len(new_pathset)

46866
18131
305995


In [98]:
# How many unique ID's don't have a pathset attacheds
len(df[df['pathnum'].isnull()])/float(len(df))

0.28141936585157684

In [99]:
# Create an indicator

In [100]:
# Now look up the probability of each path between new_pathset and pathset_paths


## Check if path exists in the pathset

In [34]:
def check_path_existence(row, pathset_df, unique_id, lookup_field):
    """
    Compare field contents from chosen paths vs entire path set,
    Return 1 if chosen path exists in pathset, 0 otherwise
    """
    
    if len(pathset_df[(pathset_df[unique_id] == row[unique_id]) & \
                        (pathset_df[lookup_field] == row[lookup_field])]) > 0:
        return 1
    else:
        return 0

In [35]:
route_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='unique_id', lookup_field='path_routes', axis=1)

In [36]:
mode_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='unique_id', lookup_field='path_modes', axis=1)

## To add: stop based data, origin, destination, and transfers, unique set of those?

In [37]:
# Mode match is the only one that should match right now because the transit route IDs aren't attached
mode_match.max()

1L

In [38]:
mode_match.mean()

0.272571838288015

In [55]:
obs[obs['unique_id'] == '100122_15336']['mode']

24810    walk_access
24811      local_bus
24812    walk_egress
Name: mode, dtype: object

In [56]:
model[model['unique_id'] == '100122_15336']['mode']

36195    walk_access
36196      local_bus
36197    walk_egress
Name: mode, dtype: object

In [51]:
mode_match = pd.DataFrame(mode_match,columns=['mode_match'])
# mode_match[mode_match['mode_match'] == 0]
mode_match[mode_match.index == '100066_15335']

,mode_match
unique_id,
100066_15335,1


In [52]:
mode_match

,mode_match
unique_id,
100066_15335,1
1000_431,0
1000_432,0
100122_15336,1
10022_3618,0
100262_15337,0
100281_15338,1
10028_3620,0
1002_433,0


In [42]:
total_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='unique_id', lookup_field='path_components', axis=1)

In [44]:
total_match

unique_id
100066_15335    0
1000_431        0
1000_432        0
100122_15336    0
10022_3618      0
100262_15337    0
100281_15338    0
10028_3620      0
1002_433        0
10036_3621      0
10038_3622      0
10040_3623      0
100476_15340    0
100488_15341    0
100495_15342    0
...
968_406      0
974_408      0
974_409      0
975_411      0
975_412      0
977_413      0
978_414      0
978_415      0
980_417      0
980_418      0
984_420      0
987_423      0
988_424      0
990_427      0
997_430      0
Length: 18131, dtype: int64

In [43]:
df = pd.DataFrame([route_match,mode_match,total_match]).T
df['id'] = df.index.get_level_values(0).values
df.columns = ['route_match','mode_match','total_match','unique_id']

ValueError: all the input array dimensions except for the concatenation axis must match exactly

,path_routes,path_modes,path_components,unique_id
unique_id,,,,
100066_15335,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335
1000_431,MUNI Caltrain_San Francisco Burlingame Tri-...,walk_access local_bus transfer commuter_rail w...,930.0 walk_access _24893.0 24893.0 local_bus M...,1000_431
1000_432,MUNI Caltrain_San Francisco Burlingame Tri-...,walk_access local_bus transfer commuter_rail w...,930.0 walk_access _24893.0 24893.0 local_bus M...,1000_432


In [60]:
new_pathset.head(3)

path_routes                         path_modes  \
unique_id    pathnum                                                  
100066_15335 0          76_14MNO  walk_access local_bus walk_egress   
             1          74_44NBL  walk_access local_bus walk_egress   
             2          74_62NBL  walk_access local_bus walk_egress   

                                                        path_components  \
unique_id    pathnum                                                      
100066_15335 0        2361 walk_access _9376 9376 local_bus 76_14MNO...   
             1        2361 walk_access _9376 9376 local_bus 74_44NBL...   
             2        2361 walk_access _9376 9376 local_bus 74_62NBL...   

                         unique_id  
unique_id    pathnum                
100066_15335 0        100066_15335  
             1        100066_15335  
             2        100066_15335

In [ ]:
# Join those results back into the observed path file
df = pd.merge(observed_path, df)

In [ ]:
# Percent of the observed trips that are found in the pathset, based on matching routes, modes, and everything
df.mean()

,path_routes_x,path_modes,path_components_x,unique_id,path_routes_y,path_components_y
0,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,76_14MNO,2361 walk_access _9376 9376 local_bus 76_14MNO...
1,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_44NBL,2361 walk_access _9376 9376 local_bus 74_44NBL...
2,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_62NBL,2361 walk_access _9376 9376 local_bus 74_62NBL...
3,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_64NBL,2361 walk_access _9376 9376 local_bus 74_64NBL...
4,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_20MRL,2361 walk_access _9376 9376 local_bus 74_20MRL...
5,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,76_1CHO,2361 walk_access _9402 9402 local_bus 76_1CHO_...
6,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,76_14MNO,2361 walk_access _9402 9402 local_bus 76_14MNO...
7,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_20MRL,2361 walk_access _9402 9402 local_bus 74_20MRL...
8,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_44NBL,2361 walk_access _9402 9402 local_bus 74_44NBL...
9,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_62NBL,2361 walk_access _9402 9402 local_bus 74_62NBL...


# Get the probability of the corresponding path from the pathset

In [ ]:
def get_path_attribute(row, pathset_df, unique_id, lookup_field, value_field):
    """
    For a chosen path, return parameter from path set.
    E.g., return path probability (value_field) for observed path, based on matching path description (lookup_field)
    
    Note: if multiple paths match parameters, return the max value from the matching set
    """
    
    if len(pathset_paths[(pathset_df[unique_id] == row[unique_id]) & \
                        (pathset_df[lookup_field] == row[lookup_field])]) > 0:
        try:
            return pathset_paths[(pathset_df[unique_id] == row[unique_id]) &  \
                             (pathset_df[lookup_field] == row[lookup_field])][value_field].values.max().index
        except:
            return 0
    else:
        return 0

In [ ]:
# new_pathset['pathnum'] = new_pathset.index.get_level_values(1).values
# new_pathset['unique_id'] = new_pathset.index.get_level_values(0).values
# new_pathset = new_pathset.reset_index(drop=True)

In [57]:
# Join the probability to the new_pathset file
newest_pathset = pd.merge(new_pathset,pathset_paths,left_on=['unique_id','pathnum'], right_on=['unique_id','pathnum'])

KeyError: 'pathnum'

In [ ]:
route_based = observed_path.apply(get_path_attribute, pathset_df=newest_pathset, unique_id='unique_id', 
                                  lookup_field='path_routes', value_field='pf_probability', axis=1)

In [ ]:
entire_path = observed_path.apply(get_path_attribute, pathset_df=newest_pathset, unique_id='unique_id', 
                                  lookup_field='path_components', value_field='pf_probability', axis=1)

In [ ]:
a = observed_path[(observed_path['unique_id'] == '1095531_3_1')]['path_components'].values

In [ ]:
b = modeled_path[(modeled_path['unique_id'] == '1095531_3_1')]['path_components'].values[0]

In [ ]:
newest_pathset[newest_pathset['path_components'] == b].to_csv('test.csv')

In [ ]:
# newest_pathset[observed_path['path_components'] == observed_path[(observed_path['unique_id'] == '1095531_3_1')]['path_components'].values[0]]

In [ ]:
route_based[route_based.index == '1095531_3_1']

In [ ]:
pathset_paths[pathset_paths['chosen'] == 1]

In [ ]:
# Convert fields to list
df['path_routes_observed_list'] = df['path_routes_observed'].apply(lambda x: filter(None,x.split(" ")))
df['path_routes_model_list'] = df['path_routes_model'].apply(lambda x: filter(None,x.split(" ")))

In [ ]:
df['path_routes_observed_list'].isin(df['path_routes_model_list'].values())

In [ ]:
df['path_routes_model_list'].values()

In [ ]:
# Store the output in a new file


# Create "path routes"
obs['path_routes'] = obs['route_id'].apply(lambda x: x.strip())
obs_path_routes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
# observed_paths.index = obs_path_routes.index
# observed_paths = pd.DataFrame(index=obs_path_routes.index)
# observed_paths['path_routes'] = obs_path_routes

In [ ]:
# produce observed_paths file



In [ ]:
# Store the output in a new file
observed_paths = pd.DataFrame()

# Create "path routes"
obs['path_routes'] = obs['route_id'].apply(lambda x: x.strip())
obs_path_routes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths.index = obs_path_routes.index
observed_paths['path_routes'] = obs_path_routes

In [ ]:
# Create "path_modes"
obs['path_modes'] = obs['mode'].apply(lambda x: x.strip())
obs_path_modes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['mode'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths['path_modes'] = obs_path_modes

In [ ]:
# Create "path_components"
obs['path_components'] = obs['A_id_num'].astype('str')+" "+obs['mode']+" "+obs['route_id']
obs['path_components'] = obs['path_components'].apply(lambda x: x.strip())
obs_path_components = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_components'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths['path_components'] = obs_path_components

In [ ]:
# We need similar fields to compare against in the modeled data
